# Задача:

Разработать алгоритмы классификации генетических мутаций на основе клинических данных (текста). Предсказать класс, в котором была генетическая мутация.

Описание файлов:

- **training_variants** - содержит описание генетических мутаций, используемых для обучения. 
Поля: 
ID - идентификатор строки, который используется для связи мутации с клиническими данными, Gene - ген, в котором находится эта генетическая мутация, Variation - изменение аминокислот для этой мутации, Class - 1-9 класс, в котором эта генетическая мутация была классифицирована
- **training_text** - файл с разделителями в виде двойной трубы (||), содержащий клинические данные (текст), используемые для классификации генетических мутаций. 
Поля: 
ID - идентификатор строки, используемой для связи клинических данных с генетической мутацией, Text - клинические данные, используемые для классификации генетической мутации)
- **test_variants** - содержит описание генетических мутаций, используемых для обучения. 
ID - идентификатор строки, который используется для связи мутации с клиническими данными, Gene - ген, в котором находится эта генетическая мутация, Variation - изменение аминокислот для этой мутации
- **test_text** - файл с разделителями в виде двойной трубы (||), содержащий клинические данные (текст), используемые для классификации генетических мутаций.  
Поля: 
ID - идентификатор строки, используемой для связи клинических данных с генетической мутацией, Text - клинические данные, используемые для классификации генетической мутации)
- **submissionSample** - пример файла отправки в правильном формате (итоговый файл, куда заливаем наши результаты)

In [ ]:
!pip install py7zr

In [ ]:
#Импортирование библиотек
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from nltk.corpus import stopwords
import nltk
from collections import Counter
import sklearn
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV, train_test_split, ShuffleSplit, KFold
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, log_loss
import pickle
import py7zr

# 1. Exploratory Data Analysis

## Загрузка данных

Тренировочные данные

In [ ]:
training_variants = pd.read_csv('/kaggle/input/msk-redefining-cancer-treatment/training_variants.zip')
training_text =pd.read_csv("/kaggle/input/msk-redefining-cancer-treatment/training_text.zip",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)

In [ ]:
training_variants.head()

In [ ]:
training_text.head()

Собираем информацию о мутациях в генах с информацией о них в научных статьях

In [ ]:
train = pd.merge(training_variants, training_text, how = 'left', on = 'ID').fillna('')
train.head()

In [ ]:
train.shape

Тестовые данные

In [ ]:
"""test_var = pd.read_csv('/kaggle/input/msk-redefining-cancer-treatment/test_variants.zip')
test_text =pd.read_csv("/kaggle/input/msk-redefining-cancer-treatment/test_text.zip",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)
test = pd.merge(test_var, test_text, how = 'left', on = 'ID').fillna('')
test.head()""";

In [ ]:
with py7zr.SevenZipFile('../input/msk-redefining-cancer-treatment/stage2_test_text.csv.7z', mode='r') as z:
    z.extractall()
with py7zr.SevenZipFile('../input/msk-redefining-cancer-treatment/stage2_test_variants.csv.7z', mode='r') as z:
    z.extractall()

test_variants = pd.read_csv("./stage2_test_variants.csv")
test_text =pd.read_csv("./stage2_test_text.csv",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)
test = pd.merge(test_variants, test_text, how = 'left', on = 'ID').fillna('')

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
df_all = pd.concat([train, test]).reset_index(drop=True)

In [ ]:
df_all

In [ ]:
#График распределения классов
my_colors = list('rgbkymc')
df_all.Class.value_counts(sort=False).plot(kind='bar', color=my_colors);

In [ ]:
df_all.Class.value_counts(sort=False) / df_all.shape[0]*100

# 2. Предобработка текстовых данных

https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing

In [ ]:
 def preprocessing_text(text):
    PUNCT_TO_REMOVE = string.punctuation
    STOPWORDS = set(stopwords.words('english'))
    
    #Приводим текст к нижнему регистру
    text = text.lower()
    
    #Удаляем символы пунктуации, они нам не нужны
    text = text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    
    #Удаление стопслов
    text = " ".join([word for word in str(text).split() if word not in STOPWORDS])
    
    #Удаление редких слов
    cnt = Counter()
    n_rare_words = 10
    RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
    text = " ".join([word for word in str(text).split() if word not in RAREWORDS])
    
    #Stemming - приводим слова к базовой форме
    #stemmer = PorterStemmer()
    #text = " ".join([stemmer.stem(word) for word in text.split()])
    
    #Lemmatization - подобно стемматизации, но гарантия того, что слово принадлежит этому языку, работает медленнее
    #lemmatizer = WordNetLemmatizer()
    #wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    #pos_tagged_text = nltk.pos_tag(text.split())    
    #text = " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
    return text

In [ ]:
df_all.loc[0]

In [ ]:
preprocessing_text(df_all.TEXT[0])[:120]

In [ ]:
%%time
df_all['clean_text'] = df_all.TEXT.apply(lambda x: preprocessing_text(x))

Функция для подсчета слова в тексте, модет использоваться в качестве дополнительного генерирования фичей

In [ ]:
def count_words(TEXT, Gene_Var):
    wordlist = TEXT.split(' ')
    cnt = 0
    for s in wordlist:
        if (Gene_Var==s):
            cnt+=1
    return cnt

In [ ]:
#Что можно еще добавить? как пример

#df_all['Gene_cnt'] = df_all.apply(lambda x: count_words(x['TEXT'], x['Gene']), axis=1)
#df_all['Variation_cnt'] = df_all.apply(lambda x: count_words(x['TEXT'], x['Variation']), axis=1)

In [ ]:
df_all

Векторизация текста

In [ ]:
def Vectorize_text(text, max_features=250, ngram_range=(1, 2), min_df=1):
    X_vect = list(text)
    tfidf = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range, max_features=max_features)
    #X_vect = tfidf.fit_transform(text).toarray() 
    X_vect = pd.DataFrame(tfidf.fit_transform(X_vect).toarray(), index=text.index)      
    return X_vect

In [ ]:
with open('df_all.pickle', 'wb') as handle:
    pickle.dump(df_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
tfidf_features = Vectorize_text(df_all['clean_text'])

In [ ]:
tfidf_features

Декомпозиция

In [ ]:
svd = TruncatedSVD(n_components=50, n_iter=5, random_state=0)
truncated_tfidf = svd.fit_transform(tfidf_features)
df_tfidf_col_name = ["tfidf_"+str(i) for i in range(50)]
df_tfidf = pd.DataFrame(truncated_tfidf)
df_tfidf.columns = df_tfidf_col_name

In [ ]:
df_tfidf[:4]

Мешок слов

In [ ]:
count_vectorizer = CountVectorizer(min_df=1, ngram_range=(1,1))
count_features = count_vectorizer.fit_transform(df_all['clean_text'])
count_svd = TruncatedSVD(n_components=50, n_iter=5, random_state=10)
count_bow = count_svd.fit_transform(count_features)
df_bow_col_name = ["bow_"+str(i) for i in range(50)]
df_bow = pd.DataFrame(count_bow)
df_bow.columns = df_bow_col_name

In [ ]:
df_bow[:4]

Закодируем Gene и Variation, чтобы привести информацию в двоичный вид

In [ ]:
df_all = pd.get_dummies(df_all, columns=['Gene', 'Variation'], drop_first=True)

Соединяем таблицы

In [ ]:
df_all = df_all.join(df_tfidf)
df_all = df_all.join(df_bow)

In [ ]:
df_all[:5]

In [ ]:
df_all.shape

In [ ]:
df_train = df_all.iloc[:train.shape[0]]
X = df_train.iloc[:,4:]
y = df_train['Class']

In [ ]:
df_test = df_all.iloc[train.shape[0]:]
X_test = df_test.iloc[:,4:]

# 3. Обучение

## XGBClassifier

Разбиваем на тренировочные данные и данные для валидации модели

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)
X_train = X.copy()
y_train = y.copy()

Делаем поиск оптимальных гиперпараметров

In [ ]:
"""
def score_func(estimator, X, y):
    score1 = log_loss(y,estimator.predict(X,
                           ntree_limit=estimator.best_ntree_limit),
                          labels=list(range(1,10)))
    return -score1

xgb = XGBClassifier(
    objective = 'multi:softprob',
    eval_metric = 'mlogloss',
    num_class = 9,
    nthread=4,
    seed=10
)

parameters = {
    'max_depth': range (4, 7, 1),
    'learning_rate': [0.1, 0.01, 0.05]
}

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=parameters,
    scoring = score_func,
    n_jobs = 10,
    cv = 3,
    verbose=True
)

grid_search.fit(X_train, y_train)""";

Классы распознаются с 0, поэтому вычитаем 1, чтобы корректно запустился алгоритм, после, мы ее прибавим к нашему предикту

In [ ]:
y_train = y_train-1

In [ ]:
xgb_param = {'objective': 'multi:softprob',
          'eval_metric' : 'mlogloss',
          'learning_rate' : 0.05,
          'max_depth' : 5,
          'num_class' : 9,
          'nthread': 4,
          'seed': 10}

dtrain_xgb = xgb.DMatrix(X_train, label=y_train)

xbg_result = xgb.cv(xgb_param, 
                    dtrain_xgb, 
                    num_boost_round=300, 
                    nfold=3,
                    stratified=True, 
                    early_stopping_rounds=50, 
                    verbose_eval=100, 
                    show_stdv=True)

Выбираем номер наилучшего раунда

In [ ]:
num_round_xgb = len(xbg_result['test-mlogloss-mean'])
print('num boost rounds xgb=' + str(num_round_xgb))

Тренируем нашу итоговую модель

In [ ]:
xgb_cl = xgb.train(xgb_param, dtrain_xgb, num_boost_round=num_round_xgb)

Подготавливаем тестовые данные, чтобы модель воспринимала формат входных тестовых данных как и у тренировочных

In [ ]:
xgtest = xgb.DMatrix(X_test)

Предикт наших тестовых данных

In [ ]:
y_pred = xgb_cl.predict(xgtest)

Создаем наш датафрейм и записываем итоговые результаты

In [ ]:
classes = ['class1', 'class2', 'class3', 'class4','class5', 'class6', 'class7', 'class8','class9']
submit = pd.DataFrame(y_pred, columns=classes)
submit['ID'] = test['ID'].values

In [ ]:
submit = submit[['ID', 'class1', 'class2', 'class3', 'class4','class5', 'class6', 'class7', 'class8','class9']]

Иииии записываем в наш файл! :)

In [ ]:
submit.to_csv('submission.csv', index=False)